In [1]:
import numpy as np
import pandas as pd
from pydataset import data
import env as env
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix
import wrangle as acq
import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


from io import StringIO

In [2]:
titanic = acq.get_titanic_data()
titanic = acq.prep_titanic(titanic)

csv file found and loaded


In [3]:
titanic.head()

,survived,pclass,sex,age,fare,embark_town
0,0,3,male,22.0,7.2500,Southampton
1,1,1,female,38.0,71.2833,Cherbourg
2,1,3,female,26.0,7.9250,Southampton
3,1,1,female,35.0,53.1000,Southampton
4,0,3,male,35.0,8.0500,Southampton


In [4]:
X = titanic.drop(columns= 'survived')
Y = titanic.survived

X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

X_train, X_validate, X_test, y_train, y_validate, y_test = acq.train_validate_test_split(X, Y)

In [5]:
baseline_prediction = y_train.mode()

In [6]:
type(baseline_prediction) #-- ?

pandas.core.frame.DataFrame

In [7]:
def establish_baseline(y_train):
    baseline_prediciton = y_train.mode()
    
    y_train_pred = pd.Series((baseline_prediciton[0]), range(len(y_train)))
    
    cm = confusion_matric(y_train, y_train_pred)
    tn, fp, fn, tp = cm.ravel()
    
    accuracy = (tp+tn+fp+fn)
    
    return 

In [8]:
# write a function to compute the baseline for a classification model

def establish_baseline(y_train):
    baseline_prediction = y_train.mode()


    y_train_pred = pd.Series((baseline_prediction[0]), range(len(y_train)))

    cm = confusion_matrix(y_train, y_train_pred)
    tn, fp, fn, tp = cm.ravel()

    accuracy = (tp+tn)/(tn+fp+fn+tp)
    return accuracy

1. Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

In [9]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_train_pred = knn.predict(X_train)

ValueError: could not convert string to float: 'male'

2. Evaluate your results using the model score, confusion matrix, and classification report.


In [ ]:
train_score =  knn.score(X_train, y_train)
train_score

In [ ]:
cm = confusion_matrix(y_train, y_train_pred)
pd.DataFrame(cm, index=['Actual 0', 'Actual 1'], 
             columns=['Pred 0', 'Pred 1'])

In [ ]:
plot_confusion_matrix(knn, X_train, y_train)

In [ ]:
print(classification_report(y_train, y_train_pred))

In [ ]:
pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))

3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
tn, fp, fn, tp = cm.ravel()

accuracy = (tp+tn)/(tp+tn+fp+fn)

true_positive_rate = (tp)/(tp+fn)
false_positive_rate = (fp)/(fp+tn)
true_negative_rate = (tn)/(fp+tn)
false_negative_rate = (fn)/(tp+fn)

precision = (tp)/(tp+fp)
recall = (tp)/(tp+fn)
f1_score = 2*(precision*recall) / (precision+recall)

support_pos = tp+fn
support_neg = fp+tn

dict = {
    'metrics': ['accuracy'
        ,'true_positive_rate'
        ,'false_positive_rate'
        ,'true_negative_rate'
        ,'false_negative_rate'
        ,'precision'
        ,'recall'
        ,'f1_score'
        ,'support_pos'
        ,'support_neg']
    ,'score': [accuracy
        ,true_positive_rate
        ,false_positive_rate
        ,true_negative_rate
        ,false_negative_rate
        ,precision
        ,recall
        ,f1_score
        ,support_pos
        ,support_neg        
    ]}
pd.DataFrame(dict)

In [ ]:
def print_cm_metrics(cm):
    tn, fp, fn, tp = cm.ravel()

    accuracy = (tp + tn)/(tn + fp + fn + tp)

    true_positive_rate = tp/(tp + fn)
    false_positive_rate = fp/(fp + tn)
    true_negative_rate = tn/(tn + fp)
    false_negative_rate = fn/(fn + tp)

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    f1_score = 2*(precision*recall)/(precision+recall)

    support_pos = tp + fn
    support_neg = fp + tn

    dict = {
        'metric' : ['accuracy'
                    ,'true_positive_rate'
                    ,'false_positive_rate'
                    ,'true_negative_rate'
                    ,'false_negative_rate'
                    ,'precision'
                    ,'recall'
                    ,'f1_score'
                    ,'support_pos'
                    ,'support_neg']
        ,'score' : [accuracy
                    ,true_positive_rate
                    ,false_positive_rate
                    ,true_negative_rate
                    ,false_negative_rate
                    ,precision
                    ,recall
                    ,f1_score
                    ,support_pos
                    ,support_neg]
    }

    return pd.DataFrame(dict)

4. Run through steps 1-3 setting k to 10

In [ ]:
def knn_fit_predict(k, X_train, y_train, X_validate):
    '''
    This function takes n_neighbors, X_train,  target  and X_val
    and returns knn, predictions for train set and validate set
    '''
    # MAKE the thing
    knn = KNeighborsClassifier(n_neighbors=k)

    # FIT the thing
    knn.fit(X_train, y_train)

    # USE the thing
    y_train_pred = knn.predict(X_train)
    y_validate_pred = knn.predict(X_validate)
    
    return knn, y_train_pred, y_validate_pred

In [ ]:
def evaluate_clf(model, X, y, y_pred):
    '''
    This function can be used on any classification model
    It takes in a model, features, target and prediction
    and returns the accuracy, confusion matrix and classification report
    '''
    # model score
    accuracy = model.score(X, y)

    # confusion matrix
    cm = confusion_matrix(y, y_pred)
    cmdf = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'], 
                       columns=['Pred 0', 'Pred 1'])

    # classification report
    crdf = pd.DataFrame(classification_report(y, y_pred, output_dict=True))
    
    # confusion matrix metrics
    metrics = print_cm_metrics(cm)
    
    return accuracy, cmdf, crdf, metrics

In [ ]:
k = 10

knn, y_train_pred, y_validate_pred = knn_fit_predict(k, X_train, y_train, X_validate)

accuracy_t, cmdf_t, crdf_t, metrics_t = evaluate_clf(knn, X_train, y_train, y_train_pred)

accuracy_v, cmdf_v, crdf_v, metrics_v = evaluate_clf(knn, X_validate, y_validate, y_validate_pred)

In [ ]:
print(f"""KNN where K = {k}

********Train Evaluation********

Accuracy: {accuracy_t}

Confusion Matrix:
{cmdf_t}

Classification Report:
{crdf_t}

Metrics: 
{metrics_t}
 
________________________________________________

********Validate Evaluation********

Accuracy: {accuracy_v}

Confusion Matrix:
{cmdf_v}

Classification Report:
{crdf_v}

Metrics: 
{metrics_v}

""")

5. Run through steps 1-3 setting k to 20

In [ ]:
k = 20

knn, y_train_pred, y_validate_pred = knn_fit_predict(k, X_train, y_train, X_validate)

accuracy_t, cmdf_t, crdf_t, metrics_t = evaluate_clf(knn, X_train, y_train, y_train_pred)

accuracy_v, cmdf_v, crdf_v, metrics_v = evaluate_clf(knn, X_validate, y_validate, y_validate_pred)


In [ ]:
print(f"""KNN where K = {k}

********Train Evaluation********

Accuracy: {accuracy_t}

Confusion Matrix:
{cmdf_t}

Classification Report:
{crdf_t}

Metrics: 
{metrics_t}
 
________________________________________________

********Validate Evaluation********

Accuracy: {accuracy_v}

Confusion Matrix:
{cmdf_v}

Classification Report:
{crdf_v}

Metrics: 
{metrics_v}

""")

6. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

Accuracy allows you to measure the number of correct predictions
Precision is used when a false positive hold sa. high consequence
Recall is used when you need to indicate missed positive predictions
F1 Score tells us how many times a model mad ea correct prediction across the entire dataset
Support is the number of samples of the true response that lies in each class of target values.



7. Which model performs best on our out-of-sample data from validate?

In [ ]:
metrics = []

for k in range(1, 21):
    knn, y_train_pred, y_val_pred = knn_fit_predict(k, X_train, y_train, X_validate)
    
    train_acc = knn.score(X_train, y_train)
    val_acc = knn.score(X_validate, y_validate)
    
    output = {"k": k,
             "train_accuracy": train_acc,
             "validate_accuracy": val_acc}
    
    metrics.append(output)
    
eval_df = pd.DataFrame(metrics)
eval_df['difference'] = eval_df['train_accuracy'] - eval_df['validate_accuracy']

eval_df

In [ ]:
eval_df.sort_values('difference')

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(eval_df.k, eval_df.train_accuracy, label='Train Accuracy')
plt.plot(eval_df.k, eval_df.validate_accuracy, label='Validate Accuracy')
plt.plot(eval_df.k, eval_df.difference, label='Accuracy Difference')
plt.ylabel('Accuracy')
plt.legend()
plt.xticks(np.arange(0,21,1))
plt.grid()
plt.show()